MT5 is an existing pre-trained language model, trained on hundreds of languages and I will adapt it to do translation task.

It uses a **sequence-to-sequence** format for language translation.

It can be adapted for low resource languages

Uses simple transformers

Simple transformers - Implementation of 'Attention is all you need paper'

Had the encoder an decorder and subsections within it:

 It has Dropout regularization in some layers and option to use GPU -

 Dropout -  to reduce overfitting and improve the generalization of a model. It involves randomly "dropping out" (i.e., setting to zero) a certain fraction of neurons or units in a neural network during each training iteration.

In [ ]:
!pip install simpletransformers

In [ ]:
import os
import pandas as pd
import warnings

# Suppress the UserWarning related to do_sample and top_p
warnings.filterwarnings("ignore", category=UserWarning, module="transformers.generation.configuration_utils")


## Data preparation
This function splits data into training and evaluation datasets in the ratio of 80:20(80% training and 20% validation)

In [ ]:
def prepare_translation_datasets(csv_filename, train_ratio=0.8):
    data_df = pd.read_csv(csv_filename)
    train_size = int(len(data_df) * train_ratio)
    train_df = data_df[:train_size]
    eval_df = data_df[train_size:]

    # Add prefixes to the data
    train_df["prefix"] = "translate Luhya to Swahili"
    eval_df["prefix"] = "translate Luhya to Swahili"

    return train_df, eval_df

In [ ]:
train_df, eval_df = prepare_translation_datasets("preprocessed_data.csv")

In [ ]:
train_df

In [ ]:
train_df.to_csv("train.tsv", sep="\t")
eval_df.to_csv("eval.tsv", sep="\t")

## Model Training

**logging.basicConfig()**:
* Configures the logging system in Python to log messages with a severity level of "INFO" and above. It's a common way to set up logging in Python for debugging.
* It means that log messages with "INFO," "WARNING," "ERROR," and "CRITICAL" severity levels will be recorded,



In [ ]:
import logging
from simpletransformers.t5 import T5Model, T5Args
#to use T5-based models and arguments for translation
logging.basicConfig(level=logging.INFO) #basically logs information
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_df = pd.read_csv("train.tsv", sep="\t").astype(str)
eval_df = pd.read_csv("eval.tsv", sep="\t").astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

You can use a prefix value to tell an mT5 (or T5) to perform a specific task. This is quite useful to train a model which can perform multiple tasks.

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 128 #maximum number of tokens or characters allowed in a sequence of text
model_args.train_batch_size = 4 #determines the number of data samples that are processed together in each iteration (or batch) during the training process.
model_args.eval_batch_size = 4
model_args.num_train_epochs = 8 #specifies the number of training epochs when training a machine learning model. An epoch is a single pass through the entire training dataset during the training process.
model_args.evaluate_during_training = True #it enables the evaluation of the model's performance on a validation dataset during the training process
model_args.evaluate_during_training_steps = 20000 # it specifies the interval at which the model's performance on a validation dataset is evaluated during the training process, measured in terms of training steps.
model_args.use_multiprocessing = False # creates multiple processes to achieve parallelism
model_args.fp16 = False
model_args.learning_rate = 3e-4
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "Luhya-Swahili Translation"
model_args.use_cuda = False

model = T5Model("mt5", "google/mt5-base", args=model_args)

In [ ]:
print(train_df.columns)

In [ ]:
train_df["prefix"] = "translate Luhya to Swahili"
train_df["input_text"] = train_df["Luhya"]
train_df["target_text"] = train_df["Swahili"]

In [ ]:
eval_df["prefix"] = "translate Luhya to Swahili"
eval_df["input_text"] = eval_df["Luhya"]
eval_df["target_text"] = eval_df["Swahili"]

In [ ]:
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/4119 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Running Epoch 7 of 8:   0%|          | 0/1030 [00:00<?, ?it/s]

  0%|          | 0/1030 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

(8240,
 {'global_step': [1030, 2060, 3090, 4120, 5150, 6180, 7210, 8240],
  'eval_loss': [4.436595710673074,
   4.850429329761239,
   4.103908126668412,
   4.065062848172446,
   4.021116838898769,
   4.3030334184336105,
   4.37587941709415,
   4.736433420994485],
  'train_loss': [5.03427791595459,
   4.277291774749756,
   2.3910603523254395,
   1.544602394104004,
   2.1327333450317383,
   0.8446611166000366,
   1.3017898797988892,
   1.428533673286438]})

In [ ]:
model

The library to calculate the Bleu score

In [ ]:
!pip install sacrebleu

In [ ]:
import sacrebleu
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
val_df = pd.read_csv("eval.tsv", sep="\t").astype(str)

luhya_truth = [eval_df.loc[eval_df["prefix"] == "translate Luhya to Swahili"]["target_text"].tolist()]
to_luhya = eval_df.loc[eval_df["prefix"] == "translate Luhya to Swahili"]["input_text"].tolist()

swahili_truth = [eval_df.loc[eval_df["prefix"] == "translate Luhya to Swahili"]["target_text"].tolist()]
to_swahili = eval_df.loc[eval_df["prefix"] == "translate Luhya to Swahili"]["input_text"].tolist()


In [ ]:
import torch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Predict
luhya_preds = model.predict(to_luhya)

swa_luh_bleu = sacrebleu.corpus_bleu(luhya_preds, luhya_truth)
print("--------------------------")
print("Swahili to Luhya: ", swa_luh_bleu.score)

swahili_preds = model.predict(to_swahili)

luh_swa_bleu = sacrebleu.corpus_bleu(swahili_preds, swahili_truth)
print("Luhya to Swahili: ", luh_swa_bleu.score)

Generating outputs:   0%|          | 0/258 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3982: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/torch/utils/hooks.py:86: UserWarning: backward hook <function TorchHistory._hook_variabl

Decoding outputs:   0%|          | 0/1030 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/hooks.py:86: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7a1e674500d0> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  warnings.warn(f"backward hook {repr(hook)} on tensor will not be "
/usr/local/lib/python3.10/dist-packages/torch/utils/hooks.py:86: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7a1e674501f0> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  warnings.warn(f"backward hook {repr(hook)} on tensor will not be "
/usr/local/lib/python3.10/dist-packages/torch/utils/hooks.py:86: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7a1e67450310> on ten

--------------------------
Swahili to Luhya:  3.0748070049807694


Generating outputs:   0%|          | 0/258 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1030 [00:00<?, ?it/s]

Luhya to Swahili:  3.0748070049807694


In [ ]:
import os

//trial

In [ ]:
import nltk
from nltk.translate import meteor_score
import pandas as pd
import sacrebleu
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import locale
from nltk import download

# Set the locale to UTF-8
locale.getdefaultlocale()
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Download the NLTK data
download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# METEOR scores
meteor_swa_luh = meteor_score.meteor_score(swahili_truth, luhya_preds)
meteor_luh_swa = meteor_score.meteor_score(luhya_truth, swahili_preds)

print("--------------------------")
print("METEOR Scores:")
print("Swahili to Luhya:")
print("METEOR Score:", meteor_swa_luh)

print("\nLuhya to Swahili:")
print("METEOR Score:", meteor_luh_swa)

--------------------------
METEOR Scores:
Swahili to Luhya:
METEOR Score: 0.000970873786407767

Luhya to Swahili:
METEOR Score: 0.000970873786407767


Calling the model from hugging face

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

model_path = "outputs"  # Replace with the path to your trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer.save_pretrained("custom_tokenizer")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


('custom_tokenizer/tokenizer_config.json',
 'custom_tokenizer/special_tokens_map.json',
 'custom_tokenizer/spiece.model',
 'custom_tokenizer/added_tokens.json',
 'custom_tokenizer/tokenizer.json')

In [ ]:
def translate_text(original_text, model, tokenizer):
    inputs = tokenizer.encode("translate Luhya to Swahili: " + original_text, return_tensors="pt")
    translation = model.generate(inputs, max_length=90, num_return_sequences=1)
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
    return translated_text


In [ ]:
original_text = "si ndareta"
translated_text = translate_text(original_text, model, tokenizer)
print("Original Text:", original_text)
print("Translated Text:", translated_text)

Original Text: si ndareta
Translated Text: nilileta


In [ ]:
original_text = "lidudu"
translated_text = translate_text(original_text, model, tokenizer)
print("Original Text:", original_text)
print("Translated Text:", translated_text)

Original Text: lidudu
Translated Text: ndege


In [ ]:
original_text = "yatsia"
translated_text = translate_text(original_text, model, tokenizer)
print("Original Text:", original_text)
print("Translated Text:", translated_text)

Original Text: yatsia
Translated Text: alienda


In [ ]:
!pip install huggingface_hub

NotImplementedError: ignored

In [ ]:
from huggingface_hub import Repository

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("")


# Save the tokenizer to the local directory
tokenizer.push_to_hub("")